Project 5: Vehicle Detection
by Lucas Hagel

In [1]:
import pickle, glob
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import importlib
from sklearn.utils import shuffle
from skimage.feature import hog
from scipy.ndimage.measurements import label
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import train_test_split

from vehicle_detection_utilities import *

%matplotlib inline

non_car_data_folder = "./data/non-vehicles/Extras/*.png"
car_data_folder     = "./data/vehicles/vehicles/*.png"

# Read in cars and non-cars
car_names     = glob.glob(car_data_folder)
non_car_names = glob.glob(non_car_data_folder)

cars = []
notcars = []

for image in car_names:
    cars.append(image)
for image in non_car_names:
    notcars.append(image)

cars = shuffle(cars)
notcars = shuffle(notcars)

#have an equal number of train and test images
minlen = min(len(cars),len(notcars))
cars    = cars[:minlen]
notcars = notcars[:minlen]


'''
print("Number of car pictures", len(cars))
print("Number of non car pictures",len(notcars))
'''
def bing():
    '''
    This function makes a sound - to be used when model training/testing is complete

    This is actually crucial for a busy person, start the NN training, and then this function
    is called (and makes the sound) when training's done
    '''
    pygame_spec = importlib.util.find_spec('pygame')
    if pygame_spec is not None:
        import pygame
        pygame.mixer.init()
        soundObj = pygame.mixer.Sound('beep1.ogg')
        soundObj.play()
        time.sleep(2)
        soundObj.stop()

/home/ron/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### have car and non-car filenames now!!!

### To do list
1. (CHECK) upload training images, equal numbers, 
2. Display 3 of each
3. Get a way to test training parameters
4. Check classification, maybe more data, maybe param tuning
5. Method checks different color spaces (be aware of their ranges)
6. Method that  checks parameters, the grid search thing

# REPEATS!!!!!

# End of REPEATS!!!!

In [2]:
iter_num = 0
logstring = ""

def get_feature_vector(image_filename, **parameters):
    return get_img_features(image_filename, **parameters)
    
search_win_params = {
'color_space' : 'YUV',
'orient' : 11,
'pix_per_cell' : 16,
'cell_per_block' : 2,
'hog_channel' : "ALL",
'fromFile' : True
}

'''
for colorspace in ['YUV', 'YCrCb', 'LUV']:
    for orient in range(6,16,4):
        for pix_per_cell in range(6,16,4):
            for hog_channel in [0,1,2,'ALL']:
'''

                
t=time.time()
car_features = [get_feature_vector(i, **search_win_params) for i in cars]
notcar_features = [get_feature_vector(i, **search_win_params) for i in notcars]
t2 = time.time()
hogtime = round(t2-t, 2)
print(round(t2-t, 2), 'Seconds to extract HOG features...')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Label definition
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

'''
Proof that train_test_split shuffles the damn thing
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
'''
rand_state = np.random.randint(0, 1000)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)


# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)

X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

logstring += "Iteration" + str(iter_num)

print(str(search_win_params['orient']),'orientations',search_win_params['pix_per_cell'],
    'pixels per cell and', search_win_params['cell_per_block'],'cells per block')
print('Feature vector length:', len(X_train[0]))

svc = LinearSVC()

t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
accuracy = round(svc.score(X_test, y_test), 4)
accuracystr = "%.4f" % accuracy
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = len(X_test)
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')


58.34 Seconds to extract HOG features...
11 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 4356
15.04 Seconds to train SVC...
Test Accuracy of SVC =  0.9619
My SVC predicts:  [ 1.  0.  1. ...,  0.  1.  0.]
For these 3517 labels:  [ 1.  0.  1. ...,  0.  1.  0.]
0.01411 Seconds to predict 3517 labels with SVC


In [4]:
import pickle
pickle.dump(svc,open('svc_long_features.p','wb') )
pickle.dump(X_scaler,open("scaler_long_features.p", 'wb'))